In [ ]:
!pip install pyrouge --upgrade
!pip install https://github.com/bheinzerling/pyrouge/archive/master.zip
!pip install pyrouge
!pip show pyrouge
!git clone https://github.com/andersjo/pyrouge.git
from pyrouge import Rouge155
!pyrouge_set_rouge_path 'pyrouge/tools/ROUGE-1.5.5'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 6.6 MB/s 
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191621 sha256=72a61c58dcaf321c70dda93d02e582311a228568034d9b6dc1a3a76b959a68a4
  Stored in directory: /root/.cache/pip/wheels/68/35/6a/ffb9a1f51b2b00fee42e7f67f5a5d8e10c67d048cda09ccd57
Successfully built pyrouge
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 202 kB 10.9 MB/s
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: pyrouge
Version: 0.1.3
Summary: A Python wrapper for the ROUGE summarization evaluation package.
Home-page: https://github.com/noutenki/pyrouge
Author: Benjamin Heinzerling, Anders Johannsen
Author-email: benjamin.heinzerling@h-its.org
License: LICENSE.txt
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: 

In [ ]:
!sudo apt-get install libxml-parser-perl

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libauthen-sasl-perl libdata-dump-perl libencode-locale-perl
  libfile-listing-perl libfont-afm-perl libhtml-form-perl libhtml-format-perl
  libhtml-parser-perl libhtml-tagset-perl libhtml-tree-perl
  libhttp-cookies-perl libhttp-daemon-perl libhttp-date-perl
  libhttp-message-perl libhttp-negotiate-perl libio-html-perl
  libio-socket-ssl-perl liblwp-mediatypes-perl liblwp-protocol-https-perl
  libmailtools-perl libnet-http-perl libnet-smtp-ssl-perl libnet-ssleay-perl
  libtimedate-perl libtry-tiny-perl liburi-perl libwww-perl
  libwww-robotrules-perl netbase perl-openssl-defaults
Suggested packages:
  libdigest-hmac-perl libgssapi-perl libcrypt-ssleay-perl libauthen-ntlm-perl
The following NEW packag

In [ ]:
%%shell
cd pyrouge/tools/ROUGE-1.5.5/data
rm WordNet-2.0.exc.db # only if exist
cd WordNet-2.0-Exceptions
rm WordNet-2.0.exc.db # only if exist

./buildExeptionDB.pl . exc WordNet-2.0.exc.db
cd ../
ln -s WordNet-2.0-Exceptions/WordNet-2.0.exc.db WordNet-2.0.exc.db

rm: cannot remove 'WordNet-2.0.exc.db': No such file or directory
adv.exc
best
better
deeper
farther
further
harder
hardest
noun.exc
aardwolves
abaci
aboideaux
aboiteaux
abscissae
acanthi
acari
acciaccature
acetabula
achaemenidae
achaemenides
acicula
aciculae
acini
acre-feet
acromia
actiniae
actinozoa
addenda
adenocarcinomata
adenomata
adieux
adyta
aecia
aecidia
aerobia
agents-general
aggiornamenti
agnomina
agones
agorae
agouties
aides-de-camp
aides-memoire
aids-de-camp
alae
alewives
alkalies
allodia
alluvia
alodia
alto-relievos
altocumuli
altostrati
alulae
alumnae
alumni
alveoli
amanuenses
ambulacra
amebae
amici_curiae
amnia
amniocenteses
amoebae
amoebiases
amoraim
amoretti
amorini
amphiarthroses
amphicia
amphimixes
amphioxi
amphisbaenae
amphorae
ampullae
amygdalae
anabases
anacolutha
anacruses
anaerobia
anagnorises
analemmata
analyses
anamneses
anamorphoses
anastomoses
anatyxes
ancones
androclinia
androecia
androsphinges
andtheridia
angelfishes
angiomata
animalcula
anlagen
annattos
a

In [ ]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pyrouge import Rouge155
import logging

In [ ]:
def read_comments(file_name):
    with open(file_name,'r') as f:
        file_content = f.readlines()
    comments = []
    for line in file_content:
        comments.append(line[:-1].split())
    return comments

In [ ]:
def build_vocab(file_name,min_num):
    with open(file_name,'r') as f:
        file_content = f.readlines()
    word_to_int = {}
    int_to_word = {}
    words_num = 0
    for line in file_content:
        line = line[:-1].split('\t')
        if int(line[2]) >= min_num:
            word_to_int[line[1]] = int(line[0])
            int_to_word[int(line[0])] = line[1] 
            words_num += 1
        else:
            break
    word_to_int['<UNK>'] = words_num+1
    int_to_word[words_num+1] = '<UNK>'
    return word_to_int,int_to_word

In [ ]:
def convert_comments(comments,word_to_int,int_to_word):
    comments_to_int = []
    for comment in comments:
        comment_to_int = [word_to_int[word] if word_to_int.get(word) != None else word_to_int['<UNK>'] for word in comment]  
        comments_to_int.append(comment_to_int)
    return comments_to_int

In [ ]:
def id_seq_to_word_seq(id_seq,id_vocab):
    return ' '.join([id_vocab[id] for id in id_seq])

In [ ]:
def prepare_evaluation(data_path,comments,int_to_word,word_to_int):
    with open(data_path,'r') as f:
        content = f.readlines()
    model_comments = {}
    for line in content:
        line = line[:-1].split('\t')
        if int(line[2]) == 1:
            model_comments[(line[0],line[1])] = [id_seq_to_word_seq(comments[int(comment)],int_to_word) for comment in line[3].split('|')]
    return model_comments

In [ ]:
def read_system_comments(data_path):
    system_comments = {}
    with open(data_path,'r') as f:
        content = f.readlines()
    for line in content:
        line = line.split('\t')
        system_comments[(line[0],line[1])] = line[2][:-1]
    return system_comments   

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/datasets.zip

Streaming output truncated to the last 5000 lines.
  inflating: datasets/matching/img/77484320.jpg  
  inflating: datasets/matching/img/77486108.jpg  
  inflating: datasets/matching/img/77490457.jpg  
  inflating: datasets/matching/img/77491438.jpg  
  inflating: datasets/matching/img/77505982.jpg  
  inflating: datasets/matching/img/77507039.jpg  
  inflating: datasets/matching/img/77516630.jpg  
  inflating: datasets/matching/img/77529116.jpg  
  inflating: datasets/matching/img/77529286.jpg  
  inflating: datasets/matching/img/77529452.jpg  
  inflating: datasets/matching/img/77529889.jpg  
  inflating: datasets/matching/img/77549446.jpg  
  inflating: datasets/matching/img/77554415.jpg  
  inflating: datasets/matching/img/77555204.jpg  
  inflating: datasets/matching/img/77564909.jpg  
  inflating: datasets/matching/img/77566475.jpg  
  inflating: datasets/matching/img/77569738.jpg  
  inflating: datasets/matching/img/77581210.jpg  
  inflating: datasets/matching/img/77581476.jpg  

In [ ]:
comments_path = '/content/drive/MyDrive/train/text.dat'
vocab_path = '/content/drive/MyDrive/train/vocab.dat'
min_num = 5

In [ ]:
comments = read_comments(comments_path)
comments

[['congrats', 'on', 'your', 'ts', '!'],
 ['thank', 'you', '!', '...', 'xoxo', '.'],
 ['thank', 'you', '!', 'xoxo', '.'],
 ['congrats', 'on', 'ts', '!', 'your', 'sets', 'are', 'amazing', '!'],
 ['absolutely', 'fabulous', 'congrats', 'on', 'tfs', '.'],
 ['congrats', 'on', 'this', 'amazing', 'top', 'set', '~', '.'],
 ['nice', '!', 'ts', 'congrats', '!'],
 ['outstanding',
  'casual',
  'outfit',
  'and',
  'stunning',
  'clothes',
  'congrats',
  'on',
  'fab',
  'top',
  'set',
  '.'],
 ['congratulations',
  'on',
  'your',
  'very',
  'pretty',
  'top',
  'fashion',
  'set',
  '.'],
 ['simple', '&', 'fab', '!', '<', '333', '.'],
 ['splendid', '!', 'congratulations', '!'],
 ['so', 'chic', 'yet', 'so', 'simplecongratzz', '.'],
 ['too', 'fashionista', 'for', 'me', '!'],
 ['adore', 'your', 'comments', ',', 'thank', 'you', '!', 'xoxo', '.'],
 ['thank',
  'you',
  'so',
  'much',
  'to',
  'everyone',
  '!',
  'i',
  'appreciate',
  'every',
  'comment',
  'and',
  'beautiful',
  'comment',
  

In [ ]:
word_to_int,int_to_word = build_vocab(vocab_path,min_num)
vocab_size = len(word_to_int)

In [ ]:
comments = convert_comments(comments,word_to_int,int_to_word)
comments

[[20, 24, 27, 73, 1],
 [23, 12, 1, 32, 120, 2],
 [23, 12, 1, 120, 2],
 [20, 24, 73, 1, 27, 56, 57, 25, 1],
 [121, 54, 20, 24, 377, 2],
 [20, 24, 8, 25, 59, 6, 147, 2],
 [50, 1, 73, 20, 1],
 [132, 100, 35, 3, 77, 547, 20, 24, 33, 59, 6, 2],
 [62, 24, 27, 37, 52, 59, 84, 6, 2],
 [170, 76, 33, 1, 34, 288, 2],
 [197, 1, 62, 1],
 [7, 44, 311, 7, 16516, 2],
 [55, 1797, 29, 64, 1],
 [103, 27, 138, 4, 23, 12, 1, 120, 2],
 [23, 12, 7, 36, 28, 295, 1, 9, 225, 301, 216, 3, 26, 216, 120, 13, 2],
 [25, 1, 20, 274, 1, 9, 234, 139, 178, 2, 13, 11, 2],
 [53, 1, 62, 16, 24, 73, 1],
 [835, 4, 2672, 2],
 [62,
  24,
  481,
  97,
  462,
  59,
  6,
  431,
  4,
  7,
  85,
  29,
  12,
  2,
  162,
  12,
  260,
  1439,
  42,
  349,
  76,
  4696,
  2],
 [3117, 1174, 1778, 1053, 377, 654, 1230, 16516, 1],
 [20, 24, 481, 97, 40, 51, 10, 2780, 51, 10, 2239, 56, 507, 2],
 [82, 457, 1027, 1, 5, 15, 2],
 [25, 3932, 2],
 [20, 24, 377, 2, 25, 2],
 [174, 1, 16516, 4, 4120, 4, 487, 16516, 16516, 1],
 [30, 1, 73, 20, 1],
 

In [ ]:
data_path = '/content/drive/MyDrive/train/newtestdata_tops.dat'
model_tops_comments = prepare_evaluation(data_path,comments,int_to_word,word_to_int)

In [ ]:
model_tops_comments

{('100007206',
  '51329747'): ['thanks ! my favorite color ( of the week , anyway ) : - ) .', 'absolutely lovely white on white set ! good luck : ) .'],
 ('100063869',
  '115809124'): ["gorgeous set ! i 'm so sorry , i accidentally liked the set without seeing the text ; please remove me from your taglist ! thank you and i 'm sorry about the <UNK> .", 'pickles ! btw love your sets they really inspire me !', 'ahh this is amazing ! i live in canada , so thanksgiving was a few weeks ago , but my <UNK> came over and we celebrated hehe ; ) .', "pickles ( only if you want to , of course ) lololol i did nothing because we do n't celebrate thanksgiving where i 'm from haha .", 'pickles : ) set is amazing ahhh blue xx .', 'pickles c : .', 'pickles please c : .'],
 ('100063869', '137623366'): ['haha , girl you rock ; d .',
  'thank you ! :3 .',
  'thanks ! do too girl ! ; p .'],
 ('100063869', '176712253'): ["i 'm glad you like it ."],
 ('100063869', '64030343'): ['good luck ! an this set is soo

In [ ]:
data_path = '/content/drive/MyDrive/train/newtestdata_downs.dat'
model_downs_comments = prepare_evaluation(data_path,comments,int_to_word,word_to_int)

In [ ]:
nfr_system_tops_comments_path = '/content/drive/MyDrive/Mini_project/matching/system comments/system_tops_comments.dat'
nfr_system_downs_comments_path = '/content/drive/MyDrive/Mini_project/matching/system comments/system_downs_comments.dat'

In [ ]:
nfr_system_tops_comments = read_system_comments(nfr_system_tops_comments_path)
nfr_system_downs_comments = read_system_comments(nfr_system_downs_comments_path)

In [ ]:
F = open('rouge/rouge_conf.xml','w')
F.write('<ROUGE-EVAL version=\"1.55\">\n')
index = 1

In [ ]:
for combination,comments in model_tops_comments.items():
    F.write('\t<EVAL ID=\"'+str(index)+'\">\n')
    F.write('\t\t<MODEL-ROOT>model</MODEL-ROOT>\n')
    F.write('\t\t<PEER-ROOT>system</PEER-ROOT>\n')
    F.write('\t\t<INPUT-FORMAT TYPE=\"SEE\">\n')
    F.write('\t\t</INPUT-FORMAT>\n')
    F.write('\t\t<PEERS>\n')
    with open('rouge/system/nfr.'+str(index)+'.txt','w') as f:
        if nfr_system_tops_comments.get(combination) != None:
            f.write(nfr_system_tops_comments[combination])
        else:
            f.write('')
    F.write('\t\t\t<P ID=\"nfr\">'+'nfr.'+str(index)+'.txt'+'</P>\n')
    F.write('\t\t</PEERS>\n')
    F.write('\t\t<MODELS>\n')
    i = 1
    for comment in comments:
        with open('rouge/model/common.'+str(i)+'.'+str(index)+'.txt','w') as f:
            f.write(comment)
        F.write('\t\t\t<M ID=\"'+str(i)+'\">'+'common.'+str(i)+'.'+str(index)+'.txt'+'</M>\n')
        i += 1
    F.write('\t\t</MODELS>\n')
    F.write('\t</EVAL>\n')
    index += 1

In [ ]:
for combination,comments in model_downs_comments.items():
    F.write('\t<EVAL ID=\"'+str(index)+'\">\n')
    F.write('\t\t<MODEL-ROOT>model</MODEL-ROOT>\n')
    F.write('\t\t<PEER-ROOT>system</PEER-ROOT>\n')
    F.write('\t\t<INPUT-FORMAT TYPE=\"SEE\">\n')
    F.write('\t\t</INPUT-FORMAT>\n')
    F.write('\t\t<PEERS>\n')
    with open('rouge/system/nfr.'+str(index)+'.txt','w') as f:
        if nfr_system_downs_comments.get(combination) != None:
            f.write(nfr_system_downs_comments[combination])
        else:
            f.write('')
    F.write('\t\t\t<P ID=\"nfr\">'+'nfr.'+str(index)+'.txt'+'</P>\n')
    F.write('\t\t</PEERS>\n')
    F.write('\t\t<MODELS>\n')
    i = 1
    for comment in comments:
        with open('rouge/model/common.'+str(i)+'.'+str(index)+'.txt','w') as f:
            f.write(comment)
        F.write('\t\t\t<M ID=\"'+str(i)+'\">'+'common.'+str(i)+'.'+str(index)+'.txt'+'</M>\n')
        i += 1
    F.write('\t\t</MODELS>\n')
    F.write('\t</EVAL>\n')
    index += 1

In [ ]:
F.write('</ROUGE-EVAL>\n')
F.close()

In [ ]:
system_dir = 'rouge/system'
model_dir = 'rouge/model'
x = logging.ERROR
r = Rouge155()
r.convert_summaries_to_rouge_format(system_dir,system_dir)
r.convert_summaries_to_rouge_format(model_dir,model_dir)

2022-06-29 02:27:45,955 [MainThread  ] [INFO ]  Processing files in rouge/system.
2022-06-29 02:27:45,959 [MainThread  ] [INFO ]  Processing nfr.9.txt.
2022-06-29 02:27:45,962 [MainThread  ] [INFO ]  Processing nfr.12.txt.
2022-06-29 02:27:45,966 [MainThread  ] [INFO ]  Processing nfr.7.txt.
2022-06-29 02:27:45,969 [MainThread  ] [INFO ]  Processing nfr.5.txt.
2022-06-29 02:27:45,971 [MainThread  ] [INFO ]  Processing nfr.6.txt.
2022-06-29 02:27:45,975 [MainThread  ] [INFO ]  Processing nfr.4.txt.
2022-06-29 02:27:45,977 [MainThread  ] [INFO ]  Processing nfr.16.txt.
2022-06-29 02:27:45,980 [MainThread  ] [INFO ]  Processing nfr.17.txt.
2022-06-29 02:27:45,982 [MainThread  ] [INFO ]  Processing nfr.3.txt.
2022-06-29 02:27:45,985 [MainThread  ] [INFO ]  Processing nfr.13.txt.
2022-06-29 02:27:45,988 [MainThread  ] [INFO ]  Processing nfr.15.txt.
2022-06-29 02:27:45,991 [MainThread  ] [INFO ]  Processing nfr.8.txt.
2022-06-29 02:27:45,994 [MainThread  ] [INFO ]  Processing nfr.14.txt.
20

In [ ]:
#ROUGE-1.5.5.pl -n 4 -2 4 -u -a rouge_conf.xml

SyntaxError: ignored